In [10]:
import pandas as pd
import numpy as np
import logging
import time
import yaml
from psycopg2 import sql
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy.utils import complex_json_handler, unpack_data
from yfpy import get_logger

from utils import data_upload
from db_psql_model import DatabaseCursor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
TEAMS_FILE = list(Path().cwd().parent.glob("**/teams.yaml"))[0]

try:
    with open(PATH) as file:
        credentials = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

auth_dir = ''
league_id = 525547
game_id = 371
game_code = "nfl"
offline = False
all_output_as_json = False
consumer_key = credentials["YFPY_CONSUMER_KEY"]
consumer_secret = credentials["YFPY_CONSUMER_SECRET"]
browser_callback = True

_auth_dir = auth_dir
_consumer_key = str(consumer_key)
_consumer_secret = str(consumer_secret)
_browser_callback = browser_callback
league_id = str(league_id)
game_id = str(game_id)
game_code = str(game_code)
offline = offline
all_output_as_json = all_output_as_json

yahoo_query = YahooFantasySportsQuery(
    auth_dir=_auth_dir,
    league_id=league_id,
    game_id=game_id,
    game_code=game_code,
    offline=offline,
    all_output_as_json=all_output_as_json,
    consumer_key=_consumer_key,
    consumer_secret=_consumer_secret,
    browser_callback=_browser_callback,
)

response = unpack_data(yahoo_query.get_all_yahoo_fantasy_game_keys())
try:
    league_keys = pd.read_csv(PATH.parent / "assests" / "game_keys.csv")
except:
    league_keys = pd.DataFrame({"game_id": np.nan, "season": np.nan})

game_keys = pd.DataFrame()
for r in response:
    row = pd.DataFrame(complex_json_handler(r["game"]), index=[0])
    game_keys = pd.concat([game_keys, row])

game_keys.reset_index(drop=True, inplace=True)
game_keys = game_keys[game_keys["season"] >= 2012]
game_keys = game_keys.merge(
    league_keys,
    how="outer",
    left_on=["game_id", "season"],
    right_on=["game_id", "season"],
)
game_keys = game_keys[
    [
        "game_id",
        "league_ID",
        "season",
        "is_game_over",
        "is_offseason"
    ]
]
game_keys.drop_duplicates(ignore_index=True, inplace=True)

game_id = list(game_keys["game_id"])
weeks = pd.DataFrame()
for g in game_id:
    response = yahoo_query.get_game_weeks_by_game_id(g)
    for r in response:
        row = pd.json_normalize(complex_json_handler(r["game_week"]))
        row["game_id"] = g
        weeks = pd.concat([weeks, row])

weeks.rename(columns={"display_name": "week"}, inplace=True)
weeks = weeks[["week", "start", "end", "game_id"]]
weeks = weeks.iloc[:, 1:]
weeks["start"] = weeks["start"].astype("datetime64[D]")
weeks["end"] = weeks["end"].astype("datetime64[D]")
weeks.drop_duplicates(ignore_index=True, inplace=True)

In [11]:
game_keys

,game_id,league_ID,season,is_game_over,is_offseason
0,273,777818,2012,1,1
1,314,731544,2013,1,1
2,331,793441,2014,1,1
3,348,570821,2015,1,1
4,359,185373,2016,1,1
5,371,525547,2017,1,1
6,380,236583,2018,1,1
7,390,215172,2019,1,1
8,399,12514,2020,1,1
9,406,103661,2021,1,1


In [12]:
weeks

,week,start,end,game_id
0,1,2012-09-05,2012-09-10,273
1,2,2012-09-11,2012-09-17,273
2,3,2012-09-18,2012-09-24,273
3,4,2012-09-25,2012-10-01,273
4,5,2012-10-02,2012-10-08,273
...,...,...,...,...
184,14,2022-12-06,2022-12-12,414
185,15,2022-12-13,2022-12-19,414
186,16,2022-12-20,2022-12-26,414
187,17,2022-12-27,2023-01-02,414


In [30]:
date = np.datetime64("2023-01-07", "D")
game_ids = weeks["game_id"][(weeks["end"] > date)]
game_id_ = game_ids.min()
week_1 = weeks[(weeks["game_id"] == game_id_) & (weeks["week"] == 1)]
season = week_1["start"].values[0].astype("datetime64[Y]").astype(int) + 1970
season

2022